In [1]:
import findspark
findspark.init()
from pyspark.ml.recommendation import ALS
from pyspark.sql.functions import col
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.sql import SparkSession


In [2]:
spark = SparkSession.builder \
    .appName("MyApp") \
    .master("local") \
    .getOrCreate()
df = spark.read.csv("a.csv", header=True, inferSchema=True)

In [3]:
train_data,val_data, test_data = df.randomSplit([0.7, 0.2,0.1], seed=12345)

In [6]:
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
from pyspark.ml.recommendation import StreamingRecommendationModel

# Khởi tạo SparkContext và StreamingContext

rank = 10
numIterations = 10
model = ALS.train(train_data, rank, numIterations)

# Tạo model streamingALS từ model ALS đã xây dựng
streaming_model = StreamingRecommendationModel(model)

ImportError: cannot import name 'StreamingRecommendationModel' from 'pyspark.ml.recommendation' (D:\spark\python\pyspark\ml\recommendation.py)

In [ ]:
# Đưa ra dự đoán cho các sự kiện mới
stream_items = stream_ratings.map(lambda rating: rating[1]).distinct()
stream_users = stream_ratings.map(lambda rating: rating[0]).distinct()
stream_new_ratings = stream_users.cartesian(stream_items).map(lambda x: (x[0], x[1]))
stream_predictions = streaming_model.predictOn(stream_new_ratings).map(lambda rating: ((rating[0], rating[1]), rating[2]))

# In ra kết quả dự đoán
stream_predictions.pprint()

# Khởi chạy StreamingContext
ssc.start()
ssc.awaitTermination()

In [5]:
als = ALS(rank=10, maxIter=10, regParam=0.01, userCol="user_id", itemCol="item_id", ratingCol="rating")

# Khởi tạo dataframe với schema (user_id, item_id, rating)
df = spark.createDataFrame([(1, 1, 5.0), (1, 2, 3.0), (2, 1, 4.0)], ["user_id", "item_id", "rating"])

In [8]:
model = als.fit(df)
U_new = model.userFactors
V_new = model.itemFactors

D:\spark\python\pyspark\sql\context.py:125: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [4]:
import numpy as np
import random
def convergence_criteria(U, V, U_new, V_new, tol=1e-3):
    # Tính độ tương đồng giữa hai ma trận factorization liên tiếp
    U_new_arr = U_new.toPandas().values
    similarity = np.abs(np.dot(U_new_arr.T, V_new) - np.dot(U.toPandas().values.T, V))
    
    # Kiểm tra xem độ tương đồng có đạt mức đủ nhỏ hay không
    if np.max(similarity) < tol:
        return True
    else:
        return False

def get_new_data():
    # Lấy một tương tác người dùng ngẫu nhiên từ cơ sở dữ liệu tương tác
    user_id = random.randint(1, 10)
    item_id = random.randint(1, 10)
    rating = random.uniform(1, 5)
    my_df = spark.createDataFrame([(user_id, item_id, rating)], schema=["user_id", "item_id","rating"])
    return my_df

In [ ]:
U = np.random
V = np.random
while True:
    # Lấy một mẫu dữ liệu đến mới
    new_data = get_new_data()
    
    # Thêm dữ liệu đến mới vào dataframe
    new_df = spark.createDataFrame([new_data], ["user_id", "item_id", "rating"])
    df = df.union(new_df)
    
    # Tính toán ma trận factorization mới
    model = als.fit(df)
    U_new = model.userFactors
    V_new = model.itemFactors
    
    # Kiểm tra tiêu chí dừng
    if convergence_criteria(U, V, U_new, V_new):
        break
    
    # Cập nhật ma trận factorization
    U, V = U_new, V_new

In [6]:
from pyspark.ml.evaluation import RegressionEvaluator

tolerance = 0.001
num_loops=10

evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="prediction")
# Khởi tạo mô hình ALS với các tham số
als = ALS(maxIter=5, regParam=0.01, userCol="user_id", itemCol="item_id", ratingCol="rating",nonnegative=True,implicitPrefs = False,
          coldStartStrategy="drop")

# Tạo mô hình ALS ban đầu bằng cách fit với dữ liệu đầu vào
initial_model = als.fit(train_data)

# Sử dụng mô hình ban đầu để đưa ra các dự đoán cho tập validation
predictions = initial_model.transform(val_data)

# Tính toán RMSE cho mô hình ban đầu
try:
    rmse = evaluator.evaluate(predictions)
except:
    rmse=0
last_rmse=rmse
print("RMSE for initial model: {}".format(rmse))

# Lặp lại quá trình huấn luyện với dữ liệu mới
for i in range(num_loops):
    # Lấy ra tập dữ liệu mới
    new_data = get_new_data()
    
    # Sử dụng mô hình hiện tại để đưa ra dự đoán cho tập dữ liệu mới
    new_predictions = initial_model.transform(new_data)
    
    # Kết hợp dữ liệu mới với dữ liệu cũ để tạo thành một tập dữ liệu lớn hơn
    combined_data = train_data.union(new_data)
    
    # Tạo mô hình mới bằng cách fit với tập dữ liệu mới hơn
    new_model = als.fit(combined_data)
    
    # Sử dụng mô hình mới để đưa ra dự đoán cho tập validation
    new_predictions = new_model.transform(val_data)
    
    # Tính toán RMSE cho mô hình mới
    try:
        rmse = evaluator.evaluate(new_predictions)
    except:
        rmse = 0
    print("RMSE for loop {}: {}".format(i+1, rmse))
    
    # Kiểm tra xem sự cải thiện của mô hình có đạt mức đủ nhỏ hay không
    if abs(rmse - last_rmse) < tolerance:
        break
    
    # Cập nhật mô hình hiện tại và RMSE
    initial_model = new_model
    last_rmse = rmse

RMSE for initial model: 1.7949632629184065
RMSE for loop 1: 1.9976266316803237
RMSE for loop 2: 1.7949632629184065
RMSE for loop 3: 0.9819193816927092
RMSE for loop 4: 2.269935398325997
RMSE for loop 5: 1.7949632629184065
RMSE for loop 6: 2.2865446382552346
RMSE for loop 7: 1.7949632629184065
RMSE for loop 8: 1.7949632629184065
